In [ ]:
#Se conecta Google Colaboratory con Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Instala pyspark en Google Colaboratory
!pip install pyspark

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
#Se importan las librerias
import pandas as pd
import numpy as np
import ast
import json
import pickle
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, explode, sum,expr, array_contains, split, substring, concat_ws, lit, udf, collect_list
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [ ]:
spark = SparkSession.builder.appName("Dataframes").getOrCreate()

In [ ]:
#Se cargan los archivos de la carpeta metadata-sitios y se compilan en un solo DataFrame

metadata_sitios=[]

for i in range(1,12):


    archivo = spark.read.json(f"/content/drive/My Drive/Google Maps/metadata-sitios/{i}.json")

    archivo = archivo.withColumn("MISC", col("MISC").cast("string"))

    metadata_sitios.append(archivo)

df_final = metadata_sitios[0]

for dataframe in metadata_sitios[1:]:

  df_final = df_final.unionByName(dataframe)


metadata_sitios=df_final

In [ ]:
# Lista de columnas a eliminar
columnas_a_eliminar = ['MISC', 'state', 'price', 'hours', 'description']

# Elimina las columnas especificadas
metadata_sitios = metadata_sitios.select([columna for columna in metadata_sitios.columns if columna not in columnas_a_eliminar])

#Pasa la columna 'category' a tipo string
metadata_sitios = metadata_sitios.withColumn("category", col("category").cast("string"))

# Filtra las filas que contienen 'restaurant' en la columna 'category'
metadata_sitios = metadata_sitios.filter(col("category").contains('restaurant'))

# Utiliza la función substring para eliminar el primer y último caracter de la columna 'category'
metadata_sitios = metadata_sitios.withColumn("category", expr("substring(category, 2, length(category)-2)"))

# Separa la columna 'address' en 3 columnas: 'address', 'city' y 'state'

split_col = split(metadata_sitios['address'], ',')
metadata_sitios = metadata_sitios.withColumn('state', substring(split_col.getItem(3), 2, 2))
metadata_sitios = metadata_sitios.withColumn('city', split_col.getItem(2))
metadata_sitios = metadata_sitios.withColumn('address', concat_ws(', ', split_col.getItem(0), split_col.getItem(1)))

# Filtrar las filas con estados 'FL', 'VA' o 'NV'
estados_seleccionados = ['FL', 'VA', 'NV']
metadata_sitios = metadata_sitios.filter(metadata_sitios['state'].isin(estados_seleccionados))

metadata_sitios.show()

+--------------------+----------+--------------------+--------------------+------------------+------------------+--------------------+--------------+--------------------+--------------------+-----+----------------+
|             address|avg_rating|            category|             gmap_id|          latitude|         longitude|                name|num_of_reviews|    relative_results|                 url|state|            city|
+--------------------+----------+--------------------+--------------------+------------------+------------------+--------------------+--------------+--------------------+--------------------+-----+----------------+
|Fresh Point Count...|       5.0|   Buffet restaurant|0x88c2e4e34f1ed78...|27.867489199999998|       -82.7029708|Fresh Point Count...|             2|[0x88c2e5593fbd02...|https://www.googl...|   FL|   Pinellas Park|
|Gormley's on the ...|       4.3|Modern French res...|0x8894b5a7a790972...|29.726948699999998|       -84.9815347|Gormley's on the ...|      

In [ ]:
#Se leen los reviews de Nevada

reviews_Nevada=[]


bandera=True

i=1

while bandera:

    try:




        archivo = spark.read.json(f"/content/drive/My Drive/Google Maps/reviews-estados/review-Nevada/{i}.json")



        reviews_Nevada.append(archivo)

        i+=1


    except:

        bandera=False


df_final = reviews_Nevada[0]

for dataframe in reviews_Nevada[1:]:

  df_final = df_final.unionByName(dataframe)


reviews_Nevada=df_final



In [ ]:
#Se leen los reviews de Florida

reviews_Florida=[]


bandera=True

i=1

while bandera:

    try:




        archivo = spark.read.json(f"/content/drive/My Drive/Google Maps/reviews-estados/review-Florida/{i}.json")



        reviews_Florida.append(archivo)

        i+=1


    except:

        bandera=False


df_final = reviews_Florida[0]

for dataframe in reviews_Florida[1:]:

  df_final = df_final.unionByName(dataframe)


reviews_Florida=df_final

In [ ]:
#Se leen los reviews de Virginia

reviews_Virginia=[]


bandera=True

i=1

while bandera:

    try:




        archivo = spark.read.json(f"/content/drive/My Drive/Google Maps/reviews-estados/review-Virginia/{i}.json")



        reviews_Virginia.append(archivo)

        i+=1


    except:

        bandera=False


df_final = reviews_Virginia[0]

for dataframe in reviews_Virginia[1:]:

  df_final = df_final.unionByName(dataframe)


reviews_Virginia=df_final

In [ ]:
# Agregar la columna 'state' con valores constantes a cada DataFrame
reviews_Nevada = reviews_Nevada.withColumn('state', lit('NV'))
reviews_Florida = reviews_Florida.withColumn('state', lit('FL'))
reviews_Virginia = reviews_Virginia.withColumn('state', lit('VA'))

# Concatenar los DataFrames
reviews_google = reviews_Nevada.union(reviews_Florida).union(reviews_Virginia)

# Lista de columnas a eliminar
columnas_a_eliminar = ['pics', 'resp']

# Elimina las columnas especificadas
reviews_google = reviews_google.select([columna for columna in reviews_google.columns if columna not in columnas_a_eliminar])

# Inicializar el analizador de sentimientos Vader
analyzer = SentimentIntensityAnalyzer()

# Definir una función UDF para aplicar el análisis de sentimientos y asignar los valores 0, 1 o 2
def vader_sentiment_analysis(text):
     # Verificar si el texto no es None o vacío
    if text and isinstance(text, str):
        compound_score = analyzer.polarity_scores(text)['compound']
        if compound_score >= 0.05:
            return '2'  # Positivo
        elif compound_score > -0.05 and compound_score < 0.05:
            return '1'  # Neutral
        else:
            return '0'  # Negativo
    else:
        return None  # Puedes ajustar esto según tu caso

# Registrar la función UDF
vader_sentiment_analysis_udf = udf(vader_sentiment_analysis, StringType())

reviews_google = reviews_google.withColumn('sentiment', vader_sentiment_analysis_udf(reviews_google['text']))


In [ ]:
reviews_google.show(truncate=False)

+-------------------------------------+------------------------------------------------+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+---------------------+-----+---------+
|gmap_id   

In [ ]:
#Se lee el archivo business de yelp

with open("/content/drive/My Drive/Yelp/business.pkl", 'rb') as archivo:

        business_yelp = pickle.load(archivo)


In [ ]:
#Elimina las columnas repetidas que tienen casi todos sus datos nulos
business_yelp=business_yelp.iloc[:,0:14]

# Lista de columnas a eliminar
columnas_a_eliminar = ['hours', 'attributes', 'is_open']

# Elimina las columnas especificadas
business_yelp = business_yelp.drop(columnas_a_eliminar, axis=1)

# Cambiar el nombre de la columna 'review_count' a 'num_of_reviews'
business_yelp= business_yelp.rename(columns={'review_count': 'num_of_reviews'})

# Filtra las filas donde la columna 'categories' contiene la cadena 'Restaurants'
business_yelp = business_yelp[business_yelp['categories'].str.contains('Restaurants', case=False, na=False)]

#Filtra los datos de los tres estados que necesitamos
business_yelp=business_yelp[(business_yelp['state']=='NV') | ((business_yelp['state']=='FL') | (business_yelp['state']=='VA'))]

business_yelp.reset_index(inplace=True,drop=True)


business_yelp.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,num_of_reviews,categories
0,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,"Food, Delis, Italian, Bakeries, Restaurants"
1,cVBxfMC4lp3DnocjYA3FHQ,Paws The Cat Cafe,10588 109 Street,Edmonton,FL,T5H 3B2,53.549633,-113.50878,5.0,20,"Coffee & Tea, Cafes, Pets, Restaurants, Pet Ad..."
2,seKihQKpGGnCeLuELRQPSQ,Twin Peaks,6880 E 82nd St,Indianapolis,FL,46250,39.906295,-86.047463,3.5,257,"Sports Bars, American (New), American (Traditi..."
3,L_TT0BFmFwORAMaA83K54A,Village Tap Room,838 Broad Ripple Ave,Indianapolis,FL,46220,39.869911,-86.143577,2.5,23,"Gastropubs, Cocktail Bars, Beer Bar, Bars, Res..."
4,8rb-3VYXE37IZix4yOdskw,Sharky's Sports Bar & Grill,820 N Black Horse Pike,Williamstown,NV,08094,39.696801,-74.999821,2.5,29,"American (Traditional), Bars, Nightlife, Sport..."


In [ ]:
#Se lee el archivo checkin de yelp

checkin_yelp=[]

with open("/content/drive/My Drive/Yelp/checkin.json","r", encoding= 'utf-8') as filejson:


                # Procesa cada línea del archivo como un objeto JSON
                for linea in filejson:

                        # Intenta cargar la línea como un objeto JSON
                        objeto = json.loads(linea)

                        checkin_yelp.append(objeto)

checkin_yelp=pd.DataFrame(checkin_yelp)


In [ ]:
business_ids= business_yelp['business_id'].tolist()

# Filtrar las filas basadas en 'business_ids'
checkin_yelp= checkin_yelp[checkin_yelp['business_id'].isin(business_ids)]

checkin_yelp.reset_index(inplace=True,drop=True)

In [ ]:
checkin_yelp.head(5)

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--ZVrH2X2QXBFdCilbirsw,"2010-08-12 18:21:29, 2010-08-20 17:49:17, 2012..."
2,--epgcb7xHGuJ-4PUeSLAw,"2010-10-01 16:31:41, 2010-12-16 19:13:47, 2011..."
3,-0EdehHjIQc0DtYU8QcAig,"2013-11-17 12:32:05, 2015-04-09 22:37:03, 2015..."
4,-0FX23yAacC4bbLaGPvyxw,"2018-04-28 00:39:07, 2018-04-28 19:23:17, 2018..."


In [ ]:
#Se lee el archivo review de yelp

review_yelp = spark.read.json("/content/drive/My Drive/Yelp/review.json")

In [ ]:
review_yelp.count()

6990280

In [ ]:
review_yelp = review_yelp.withColumn('sentiment', vader_sentiment_analysis_udf(review_yelp['text']))

# Filtrar las filas en review_yelp donde 'business_id' está en la lista
review_yelp = review_yelp.filter(col('business_id').isin(business_ids))

In [ ]:
review_yelp.count()

1081517

In [ ]:
#Se lee el archivo tip de yelp

tip_yelp=[]

with open("/content/drive/My Drive/Yelp/tip.json","r", encoding= 'utf-8') as filejson:


                # Procesa cada línea del archivo como un objeto JSON
                for linea in filejson:

                        # Intenta cargar la línea como un objeto JSON
                        objeto = json.loads(linea)

                        tip_yelp.append(objeto)

tip_yelp=pd.DataFrame(tip_yelp)


In [ ]:
# Filtrar las filas basadas en 'business_ids'
tip_yelp= tip_yelp[tip_yelp['business_id'].isin(business_ids)]

tip_yelp.reset_index(inplace=True,drop=True)

In [ ]:
tip_yelp.head(5)

,user_id,business_id,text,date,compliment_count
0,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
1,OttfcRxgRrYsTg9EV5Aozg,clwjLY7PdYJpe7IP9lrqEw,Order the Tortilla Soup,2014-06-17 01:20:14,0
2,j2sEA3hiUcwHfq9Ml6M47g,EXYbKA1tocvOK_1tXxZXLA,Don't go for dinner. They close at 6. Really Y...,2011-10-13 03:15:15,0
3,kjFgyrCvmVVGSlgWzRXILw,ReX09lhufLTAx19krkltDA,Got the grilled chicken entree. It was delicio...,2013-06-10 20:18:41,0
4,BC2LVfzoizlY4kJPzi_13Q,tkDs-InBqql7rqlAan97vA,"Looks like mini burgers, similar to white castle",2012-07-28 14:35:32,0


In [ ]:
#Se lee el archivo user de yelp

user_yelp= pd.read_parquet("/content/drive/My Drive/Yelp/user.parquet")

In [ ]:
user_yelp.head(5)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0
